In [65]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import random

In [61]:
#import the data
train_x_dum = pd.read_csv("train_dummy_final.csv")

test_x_dum = pd.read_csv("test_dummy_final.csv")
test_x_ids = test_x_dum['id']


#create x/y split for trianing 
train_y_dum = pd.DataFrame(train_x_dum['status_group'])
train_x_dum = train_x_dum.drop(['status_group'], axis = 1)

#drop permit and public meeting
train_x_dum = train_x_dum.drop(['public_meeting', 'permit'], axis = 1)
test_x_dum = test_x_dum.drop(['public_meeting', 'permit'], axis = 1)

# # set ID as index for train x and y 
train_x_dum = train_x_dum.set_index(['id'])
test_x_dum = test_x_dum.set_index(['id'])

#set data type for non int numbers
train_x_dum = train_x_dum.astype({"amount_tsh": int})
test_x_dum = test_x_dum.astype({"amount_tsh": int})

#split data just for testing 
X_train_dum, X_test_dum, Y_train_dum, Y_test_dum = train_test_split(train_x_dum, train_y_dum['status_group'], train_size=0.6, random_state=42)

{'classification__max_depth': [1],
 'classification__n_estimators': [50],
 'classification__learning_rate': [1]}

In [86]:
params = {'max_depth':[3,5,7],
          'n_estimators':[50,100,250,500],
          'learning_rate':[0.1,0.25,0.5,0.75,1]}

new_params = {'gradientboostingclassifier__' + key: params[key] for key in params}

xgb = GradientBoostingClassifier()
smote = SMOTE(random_state=20)

xgb_pipeline = make_pipeline(smote, xgb)

grid_imba = GridSearchCV(xgb_pipeline, param_grid=new_params, n_jobs=-1, verbose=300)

grid_imba.fit(X_train_dum, Y_train_dum)

grid_imba.best_params_
y_pred = grid_imba.predict(X_test_dum)
print(classification_report(Y_test_dum, y_pred))

Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 

In [98]:
predictions = pd.DataFrame(test_x_ids)
for i in range(6):
    #full model    
    randomseedx = random.randint(1,10000)

    model_full = GradientBoostingClassifier(max_depth = 7,
                                            n_estimators = 500,
                                           learning_rate = .1,
                                           random_state = randomseedx)

    model_full.fit(train_x_dum, train_y_dum.values.flatten())
    #Create predictions
    preds_class_full = pd.DataFrame(model_full.predict(test_x_dum))
    names = 'Model ' + str(i+1)
    predictions[names] = preds_class_full
    print(names + " Has completed")



predictions

Model 1 Has completed
Model 2 Has completed
Model 3 Has completed
Model 4 Has completed
Model 5 Has completed
Model 6 Has completed


,id,Model 1,Model 2,Model 3,Model 4,Model 5,Model 6
0,50785,0,0,0,0,0,0
1,51630,0,0,0,0,0,0
2,17168,0,0,0,0,0,0
3,45559,2,2,2,2,2,2
4,49871,0,0,0,0,0,0
...,...,...,...,...,...,...,...
14845,39307,2,2,2,2,2,2
14846,18990,0,0,0,0,0,0
14847,28749,2,2,2,2,2,2
14848,33492,0,0,0,0,0,0


In [103]:
replace_map2 = {2:"non functional", 
              1:"functional needs repair",
              0:"functional"}

predictions = predictions.replace(replace_map2)
predictions.to_csv("xgb_submision1.csv", index=False)


In [71]:
#grid search for XGBoost
#xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5, .fit(X_train_dum, Y_train_dum)
#xgb.score(X_test_dum, Y_test_dum)

params = {'max_depth':[3,5,7],
          'n_estimators':[50,100,250,500],
          'learning_rate':[0.1,0.25,0.5,0.75,1]}

xgb = GradientBoostingClassifier()
scorer = make_scorer(accuracy_score)
xgb_grid = GridSearchCV(estimator=xgb, param_grid=params, scoring=scorer, cv=5)



In [53]:
#xgb_grid.fit(X_train_dum, Y_train_dum)
#best_param = xgb_grid.best_params_
#best_param

In [ ]:
#send best param to 
pd.DataFrame.from_dict(best_param, orient='index').to_csv('best_param_xbg.csv')

In [ ]:
#fit xbg model using best params
xgb_full = GradientBoostingClassifier(n_estimators=, learning_rate=, max_depth=, .fit(train_x_dum))

In [57]:
#make predictions
xgb_preds = pd.DataFrame(grid_imba1.predict(test_x_dum))

xgb_submission = pd.concat([test_x_ids, xgb_preds], axis=1)
xgb_submission.columns = ['id', 'status_group']


replace_map2 = {2:"non functional", 
              1:"functional needs repair",
              0:"functional"}

xgb_submission['status_group'] = xgb_submission['status_group'].replace(replace_map2)
xgb_submission.to_csv("xgb_submision.csv", index=False)

In [ ]:

#submission['status_group'] = submission['status_group'].replace(replace_map2)
#submission.to_csv("submision.csv", index=False)